In [23]:
# используемые библиотеки в решении
import pandas as pd
import lightgbm

# чтение данных
X_train_in = pd.read_csv("X_train.csv", index_col=0)
y_train_in = pd.read_csv("y_train.csv", index_col=0)
X_validation = pd.read_csv("X_test.csv", index_col=0)

stud_info = pd.read_csv("studs_info.csv", index_col=0)

X_train_in["mark"] = y_train_in["mark"]


In [24]:
X_train_in

,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА,mark
51081,175711,38.03.02 Менеджмент,2018-2019,2.0,4.0,Экзамен,Маркетинг,5
72921,100647,31.05.01 Лечебное дело,2015-2016,3.0,5.0,Экзамен,"Пропедевтика внутренних болезней, лучевая диаг...",5
96438,199311,31.05.03 Стоматология,2019-2020,1.0,1.0,Экзамен,Химия,5
273044,162002,38.03.02 Менеджмент,2017-2018,2.0,3.0,Экзамен,Статистика,4
371309,108384,31.05.01 Лечебное дело,2018-2019,5.0,9.0,Экзамен,Педиатрия,3
...,...,...,...,...,...,...,...,...
224856,176141,40.03.01 Юриспруденция,2017-2018,1.0,2.0,Курсовая работа,Конституционное право,3
260998,182794,31.05.01 Лечебное дело,2018-2019,2.0,4.0,Экзамен,Нормальная физиология,3
57245,162391,31.05.01 Лечебное дело,2016-2017,1.0,2.0,Дифференцированный зачет,Иностранный язык,4
218529,202479,38.03.01 Экономика,2017-2018,1.0,1.0,Дифференцированный зачет,Информатика,4


In [25]:
stud_info

,number,Пол,Статус,Дата выпуска,Категория обучения,Форма обучения,Шифр,направление (специальность),,Образование,Дата выдачи,Что именно закончил
STD_ID,,,,,,,,,,,,
99136,13951.0,Ж,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,2629.0,СР,2013-06-26 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
99112,13998.0,Ж,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,8240.0,СР,2012-06-23 00:00:00,"МОУ СОШ N 2, ПЕСТОВО"
99109,13999.0,Ж,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,2314.0,СР,2013-06-25 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
99125,13953.0,Ж,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,2345.0,СР,2013-06-23 00:00:00,"Гимназия ""ГАРМОНИЯ"""
100676,13950.0,М,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,2650.0,СР,2012-06-29 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
...,...,...,...,...,...,...,...,...,...,...,...,...
100678,978.0,Ж,СТ,NaN,БП,Д,31.05.01,Лечебное дело,2388.0,СР,2013-06-21 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
100663,1021.0,Ж,СТ,NaN,ДОГ,Д,31.05.01,Лечебное дело,2554.0,СР,2013-06-27 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
191630,6447.0,Ж,СТ,NaN,ДОГ,Д,31.05.01,Лечебное дело,6692.0,СР,2014-05-31 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА


In [26]:

from sklearn import preprocessing

str_fields = ["НАПРАВЛЕНИЕ", "ГОД", "АТТЕСТАЦИЯ", "ДИСЦИПЛИНА"]
# str_fields = {"НАПРАВЛЕНИЕ": "study_type", "ГОД": "year", "АТТЕСТАЦИЯ": "work_type", "ДИСЦИПЛИНА": "lesson_type"}
le_s = [preprocessing.LabelEncoder() for _ in str_fields]

# for (field_name, new_field_name), le in zip(str_fields.items(), le_s):
for field_name, le in zip(str_fields, le_s):
    le.fit(((pd.concat([X_train_in[field_name], X_validation[field_name]]))))

    X_train_in[field_name] = le.transform(X_train_in[field_name])
    X_validation[field_name] = le.transform(X_validation[field_name])

for field_name in ("КУРС", "СЕМЕСТР"):
    X_train_in[field_name] = X_train_in[field_name].astype(int)
    X_validation[field_name] = X_validation[field_name].astype(int)

# вычисление средней оценки
stud_mean_mark = X_train_in.groupby("STD_ID").mean()["mark"]
X_train_in["stud_mean"] = X_train_in["STD_ID"].map(stud_mean_mark)

X_validation["stud_mean"] = X_validation["STD_ID"].map(stud_mean_mark)


In [27]:
X_train_in

,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА,mark,stud_mean
51081,175711,36,5,2,4,4,408,5,4.181818
72921,100647,24,2,3,5,4,946,5,4.116279
96438,199311,25,6,1,1,4,1326,5,4.750000
273044,162002,36,4,2,3,4,1100,4,4.172414
371309,108384,24,5,5,9,4,713,3,3.542857
...,...,...,...,...,...,...,...,...,...
224856,176141,40,4,1,2,2,364,3,3.615385
260998,182794,24,5,2,4,4,556,3,4.052632
57245,162391,24,3,1,2,0,228,4,4.458333
218529,202479,35,4,1,1,0,238,4,4.187500


In [28]:
X_train_in.sort_values(by=["stud_mean"])

,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА,mark,stud_mean
246309,164757,52,5,2,4,4,275,3,3.0
194911,192224,24,6,2,3,4,1301,3,3.0
161856,194119,40,5,1,1,0,1301,3,3.0
223307,162077,10,4,1,2,0,260,3,3.0
198063,192223,24,6,2,3,4,89,3,3.0
...,...,...,...,...,...,...,...,...,...
35943,182012,16,5,2,4,0,771,5,5.0
190139,193882,46,5,1,2,0,228,5,5.0
249056,173733,49,5,2,4,0,857,5,5.0
68595,114316,47,5,5,9,0,803,5,5.0


In [29]:

from sklearn.model_selection import train_test_split
# разделение на обучающую и тестовую выборку для проверки решения
X_train, X_test, y_train, y_test = train_test_split(X_train_in, X_train_in["mark"], shuffle=True, test_size=0.2)

# выбор параметров на основе которых будет обучение
train_col = ["STD_ID", "КУРС", "СЕМЕСТР", "stud_mean", "НАПРАВЛЕНИЕ"]

# создание и обучение модели на основе бустинга рещающих деревьев
lgb = lightgbm.LGBMRegressor(n_estimators=1000, verbose=1)

lgb.fit(X_train[train_col], y_train)


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.191278 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 585
[LightGBM] [Info] Number of data points in the train set: 84046, number of used features: 5
[LightGBM] [Info] Start training from score 4.184030


LGBMRegressor(n_estimators=1000, verbose=1)

In [30]:

from sklearn.metrics import mean_absolute_error, r2_score

# вычисление метрик получившейся модели
print(mean_absolute_error(y_test, lgb.predict(X_test[train_col])))
print(r2_score(y_test, lgb.predict(X_test[train_col])))

# сохранение решения в нужном формате для отпавки
y_pred = pd.read_csv("sample_submission.csv", index_col=0)
y_pred["mark"] = lgb.predict(X_validation[train_col])

y_pred.to_csv("baseline_submission.csv")


0.45193769289345304
0.4356079227334452
